While the original exercise uses the numpy built methods from the previous exercise to construct a dino-name generator, I will try to use Keras.

In [84]:
import numpy as np
import keras
from keras.models import Model
from keras.layers import Input, SimpleRNN, Dense, Lambda, LSTM, Masking, TimeDistributed
from keras import backend as K

How many charactars in the our data?

In [70]:
data = open('dinos.txt', 'r').read()
data= data.lower()
chars = list(set(data))
data_size, vocab_size = len(data), len(chars)
print('There are %d total characters and %d unique characters in your data.' % (data_size, vocab_size))

There are 19909 total characters and 27 unique characters in your data.


In [71]:
char_to_ix = { ch:i for i,ch in enumerate(sorted(chars)) }
ix_to_char = { i:ch for i,ch in enumerate(sorted(chars)) }
print(ix_to_char)

{0: '\n', 1: 'a', 2: 'b', 3: 'c', 4: 'd', 5: 'e', 6: 'f', 7: 'g', 8: 'h', 9: 'i', 10: 'j', 11: 'k', 12: 'l', 13: 'm', 14: 'n', 15: 'o', 16: 'p', 17: 'q', 18: 'r', 19: 's', 20: 't', 21: 'u', 22: 'v', 23: 'w', 24: 'x', 25: 'y', 26: 'z'}


Let's create our training set:

In [72]:
# open the file again, and read the lines
with open("dinos.txt") as f:
    examples = f.readlines()
examples = [x.lower().strip() for x in examples]
# Shuffle list of all dinosaur names
np.random.seed(210)
np.random.shuffle(examples)

In [73]:
# compute the max length of a word in data
maxLen = len(max(examples, key=len))
print(maxLen)

26


In [74]:
m = len(examples)
feature_size = 1  # we use the actual index, and not 1-hot-encoding

X = np.zeros((m, maxLen, feature_size))
Y = np.zeros((m, maxLen, feature_size))

for i in range(m):
    word = examples[i]
    l = len(word)        
    X[i, 0:l, :] = np.array([char_to_ix[ch] for ch in word]).reshape(-1, 1)
    Y[i, 0:l, :] = np.concatenate([X[i, 1:l, :], [[char_to_ix["\n"]]]])

In [174]:
# create the model
def simple_model(input_shape, n_a = 50):
    inp = Input(shape=input_shape, name='x')
    X = Masking(mask_value=0)(inp)
    X = SimpleRNN(n_a, return_sequences=True, name='rnn_cell')(X)
    X = Dense(vocab_size, activation='softmax', name='dense_output')(X)
    model = Model(inputs=inp, outputs=X)
    return model

In [175]:
model = simple_model(input_shape=(maxLen, feature_size))
# Use sparse loss due to not using 1-hot-encoding
model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['sparse_categorical_accuracy'])

In [176]:
from keras.callbacks import ReduceLROnPlateau
learning_rate_reduction = ReduceLROnPlateau(monitor='sparse_categorical_accuracy', patience=4, verbose=1, factor=0.5, min_lr=0.00001)

model.fit(X, Y, epochs = 10, batch_size = 64, callbacks=[learning_rate_reduction])

Epoch 1/10
1536/1536 [==============================] - 1s 828us/step - loss: 3.0801 - sparse_categorical_accuracy: 0.1036
Epoch 2/10
1536/1536 [==============================] - 1s 351us/step - loss: 2.6904 - sparse_categorical_accuracy: 0.2033
Epoch 3/10
1536/1536 [==============================] - 1s 342us/step - loss: 2.6113 - sparse_categorical_accuracy: 0.2195
Epoch 4/10
1536/1536 [==============================] - 1s 344us/step - loss: 2.5733 - sparse_categorical_accuracy: 0.2593
Epoch 5/10
1536/1536 [==============================] - 1s 345us/step - loss: 2.5419 - sparse_categorical_accuracy: 0.3029
Epoch 6/10
1536/1536 [==============================] - 1s 343us/step - loss: 2.5107 - sparse_categorical_accuracy: 0.3105
Epoch 7/10
1536/1536 [==============================] - 1s 343us/step - loss: 2.4794 - sparse_categorical_accuracy: 0.3116
Epoch 8/10
1536/1536 [==============================] - 1s 360us/step - loss: 2.4430 - sparse_categorical_accuracy: 0.3158
Epoch 9/10
1536/

We got 47% accuracy after 100-200 epochs.

Now let's build an "inference" generator:

In [178]:
def get_inference_model(model):
    rnn = model.get_layer(name='rnn_cell')
    dense = model.get_layer(name='dense_output')
    inp_infer = Input(shape=(1, feature_size))
    X = rnn(inp_infer)
    X = dense(X)
    model_infer = Model(inp_infer, X)
    return model_infer

In [179]:
inf_model = get_inference_model(model)

In [295]:
def generate_name(model, seq_len=20):  # max length of a name - 20 chars
    x = np.random.randint(1, vocab_size, size=(1, 1, 1))
    name = []
    for _ in range(seq_len):
        name.append(ix_to_char[x[0][0][0]])
        prob = model.predict(x)  # get the softmax output, handle as a distribution
        pred = np.random.choice(np.arange(vocab_size), p=prob.ravel())  # sample from the distribution 
        x = pred.reshape(1,1,1)
        if x[0][0][0] == 0:  # if we reach a new line, we break
            break
    return "".join(name)

In [308]:
generate_name(inf_model)

'sirafrssoeoaeaaurzia'

Let's try LSTM:

In [258]:
# create the model
def lstm_model(input_shape, n_a = 128):
    inp = Input(shape=input_shape, name='x')
    X = Masking(mask_value=0)(inp)
    X = LSTM(n_a, return_sequences=True, name='lstm_cell')(X)
    X = Dense(vocab_size, activation='softmax', name='dense_output_lstm')(X)
    model = Model(inputs=inp, outputs=X)
    return model

In [259]:
lstm = lstm_model(input_shape=(maxLen, feature_size))
lstm.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['sparse_categorical_accuracy'])

In [260]:
lstm.fit(X, Y, epochs = 100, batch_size = 128)

Epoch 1/100
1536/1536 [==============================] - 2s 1ms/step - loss: 3.0489 - sparse_categorical_accuracy: 0.1099
Epoch 2/100
1536/1536 [==============================] - 1s 332us/step - loss: 2.7244 - sparse_categorical_accuracy: 0.1786
Epoch 3/100
1536/1536 [==============================] - 0s 320us/step - loss: 2.6197 - sparse_categorical_accuracy: 0.2141
Epoch 4/100
1536/1536 [==============================] - 1s 340us/step - loss: 2.5324 - sparse_categorical_accuracy: 0.2386
Epoch 5/100
1536/1536 [==============================] - 0s 312us/step - loss: 2.4692 - sparse_categorical_accuracy: 0.2609
Epoch 6/100
1536/1536 [==============================] - 0s 306us/step - loss: 2.4135 - sparse_categorical_accuracy: 0.2976
Epoch 7/100
1536/1536 [==============================] - 0s 306us/step - loss: 2.3677 - sparse_categorical_accuracy: 0.3162
Epoch 8/100
1536/1536 [==============================] - 0s 308us/step - loss: 2.3286 - sparse_categorical_accuracy: 0.3329
Epoch 9/10

1536/1536 [==============================] - 0s 312us/step - loss: 1.9238 - sparse_categorical_accuracy: 0.4372
Epoch 67/100
1536/1536 [==============================] - 1s 326us/step - loss: 1.9185 - sparse_categorical_accuracy: 0.4378
Epoch 68/100
1536/1536 [==============================] - 0s 311us/step - loss: 1.9160 - sparse_categorical_accuracy: 0.4389
Epoch 69/100
1536/1536 [==============================] - 0s 315us/step - loss: 1.9114 - sparse_categorical_accuracy: 0.4410
Epoch 70/100
1536/1536 [==============================] - 0s 313us/step - loss: 1.9065 - sparse_categorical_accuracy: 0.4414
Epoch 71/100
1536/1536 [==============================] - 0s 321us/step - loss: 1.9016 - sparse_categorical_accuracy: 0.4425
Epoch 72/100
1536/1536 [==============================] - 0s 325us/step - loss: 1.8982 - sparse_categorical_accuracy: 0.4431
Epoch 73/100
1536/1536 [==============================] - 0s 318us/step - loss: 1.8962 - sparse_categorical_accuracy: 0.4465
Epoch 74/100


In [261]:
def get_inference_model_lstm(model):
    lstm = model.get_layer(name='lstm_cell')
    dense = model.get_layer(name='dense_output_lstm')
    inp_infer = Input(shape=(1, feature_size))
    X = lstm(inp_infer)
    X = dense(X)
    model_infer = Model(inp_infer, X)
    return model_infer

In [262]:
lstm_inf_model = get_inference_model_lstm(lstm)

In [312]:
generate_name(lstm_inf_model)

'zlaroiahuhiyiipiyiea'

Let's try to use an even more complex model

In [269]:
def complex_model(input_shape, n_a = 128):
    inp = Input(shape=input_shape, name='x')
    X = Masking(mask_value=0)(inp)
    X = LSTM(n_a, return_sequences=True, name='lstm_1')(X)
    X = LSTM(n_a//2, return_sequences=True, name='lstm_2')(X)
    X = LSTM(n_a//3, return_sequences=True, name='lstm_3')(X)
    X = Dense(vocab_size, activation='softmax', name='dense_output_lstm')(X)
    model = Model(inputs=inp, outputs=X)
    return model

In [270]:
complx = complex_model(input_shape=(maxLen, feature_size))

In [272]:
complx.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['sparse_categorical_accuracy'])

In [278]:
complx.fit(X, Y, epochs = 500, batch_size = 256)

Epoch 1/500
1536/1536 [==============================] - 1s 552us/step - loss: 1.6186 - sparse_categorical_accuracy: 0.5223
Epoch 2/500
1536/1536 [==============================] - 1s 516us/step - loss: 1.6152 - sparse_categorical_accuracy: 0.5221
Epoch 3/500
1536/1536 [==============================] - 1s 497us/step - loss: 1.6135 - sparse_categorical_accuracy: 0.5237
Epoch 4/500
1536/1536 [==============================] - 1s 478us/step - loss: 1.6115 - sparse_categorical_accuracy: 0.5240
Epoch 5/500
1536/1536 [==============================] - 1s 473us/step - loss: 1.6107 - sparse_categorical_accuracy: 0.5247
Epoch 6/500
1536/1536 [==============================] - 1s 475us/step - loss: 1.6083 - sparse_categorical_accuracy: 0.5263
Epoch 7/500
1536/1536 [==============================] - 1s 460us/step - loss: 1.6066 - sparse_categorical_accuracy: 0.5257
Epoch 8/500
1536/1536 [==============================] - 1s 458us/step - loss: 1.6043 - sparse_categorical_accuracy: 0.5271
Epoch 9/

1536/1536 [==============================] - 1s 484us/step - loss: 1.5021 - sparse_categorical_accuracy: 0.5562
Epoch 67/500
1536/1536 [==============================] - 1s 491us/step - loss: 1.5011 - sparse_categorical_accuracy: 0.5556
Epoch 68/500
1536/1536 [==============================] - 1s 475us/step - loss: 1.5008 - sparse_categorical_accuracy: 0.5581
Epoch 69/500
1536/1536 [==============================] - 1s 486us/step - loss: 1.5005 - sparse_categorical_accuracy: 0.5555
Epoch 70/500
1536/1536 [==============================] - 1s 467us/step - loss: 1.4974 - sparse_categorical_accuracy: 0.5587
Epoch 71/500
1536/1536 [==============================] - 1s 490us/step - loss: 1.4929 - sparse_categorical_accuracy: 0.5582
Epoch 72/500
1536/1536 [==============================] - 1s 457us/step - loss: 1.4925 - sparse_categorical_accuracy: 0.5590
Epoch 73/500
1536/1536 [==============================] - 1s 469us/step - loss: 1.4924 - sparse_categorical_accuracy: 0.5600
Epoch 74/500


1536/1536 [==============================] - 1s 476us/step - loss: 1.3884 - sparse_categorical_accuracy: 0.5879
Epoch 132/500
1536/1536 [==============================] - 1s 482us/step - loss: 1.3869 - sparse_categorical_accuracy: 0.5875
Epoch 133/500
1536/1536 [==============================] - 1s 467us/step - loss: 1.3859 - sparse_categorical_accuracy: 0.5880
Epoch 134/500
1536/1536 [==============================] - 1s 477us/step - loss: 1.3858 - sparse_categorical_accuracy: 0.5887
Epoch 135/500
1536/1536 [==============================] - 1s 461us/step - loss: 1.3823 - sparse_categorical_accuracy: 0.5895
Epoch 136/500
1536/1536 [==============================] - 1s 463us/step - loss: 1.3856 - sparse_categorical_accuracy: 0.5880
Epoch 137/500
1536/1536 [==============================] - 1s 467us/step - loss: 1.3835 - sparse_categorical_accuracy: 0.5889
Epoch 138/500
1536/1536 [==============================] - 1s 470us/step - loss: 1.3797 - sparse_categorical_accuracy: 0.5896
Epoch 

1536/1536 [==============================] - 1s 455us/step - loss: 1.2879 - sparse_categorical_accuracy: 0.6153
Epoch 197/500
1536/1536 [==============================] - 1s 467us/step - loss: 1.2837 - sparse_categorical_accuracy: 0.6188
Epoch 198/500
1536/1536 [==============================] - 1s 459us/step - loss: 1.2821 - sparse_categorical_accuracy: 0.6160
Epoch 199/500
1536/1536 [==============================] - 1s 467us/step - loss: 1.2839 - sparse_categorical_accuracy: 0.6174
Epoch 200/500
1536/1536 [==============================] - 1s 469us/step - loss: 1.2821 - sparse_categorical_accuracy: 0.6179
Epoch 201/500
1536/1536 [==============================] - 1s 466us/step - loss: 1.2789 - sparse_categorical_accuracy: 0.6191
Epoch 202/500
1536/1536 [==============================] - 1s 467us/step - loss: 1.2807 - sparse_categorical_accuracy: 0.6180
Epoch 203/500
1536/1536 [==============================] - 1s 472us/step - loss: 1.2809 - sparse_categorical_accuracy: 0.6189
Epoch 

1536/1536 [==============================] - 1s 468us/step - loss: 1.1952 - sparse_categorical_accuracy: 0.6437
Epoch 262/500
1536/1536 [==============================] - 1s 465us/step - loss: 1.1923 - sparse_categorical_accuracy: 0.6423
Epoch 263/500
1536/1536 [==============================] - 1s 471us/step - loss: 1.1924 - sparse_categorical_accuracy: 0.6424
Epoch 264/500
1536/1536 [==============================] - 1s 469us/step - loss: 1.1892 - sparse_categorical_accuracy: 0.6446
Epoch 265/500
1536/1536 [==============================] - 1s 458us/step - loss: 1.1835 - sparse_categorical_accuracy: 0.6471
Epoch 266/500
1536/1536 [==============================] - 1s 466us/step - loss: 1.1838 - sparse_categorical_accuracy: 0.6457
Epoch 267/500
1536/1536 [==============================] - 1s 461us/step - loss: 1.1834 - sparse_categorical_accuracy: 0.6458
Epoch 268/500
1536/1536 [==============================] - 1s 459us/step - loss: 1.1860 - sparse_categorical_accuracy: 0.6433
Epoch 

1536/1536 [==============================] - 1s 458us/step - loss: 1.1060 - sparse_categorical_accuracy: 0.6714
Epoch 327/500
1536/1536 [==============================] - 1s 463us/step - loss: 1.1051 - sparse_categorical_accuracy: 0.6704
Epoch 328/500
1536/1536 [==============================] - 1s 464us/step - loss: 1.1048 - sparse_categorical_accuracy: 0.6705
Epoch 329/500
1536/1536 [==============================] - 1s 478us/step - loss: 1.1038 - sparse_categorical_accuracy: 0.6714
Epoch 330/500
1536/1536 [==============================] - 1s 469us/step - loss: 1.1025 - sparse_categorical_accuracy: 0.6725
Epoch 331/500
1536/1536 [==============================] - 1s 471us/step - loss: 1.1004 - sparse_categorical_accuracy: 0.6731
Epoch 332/500
1536/1536 [==============================] - 1s 475us/step - loss: 1.0999 - sparse_categorical_accuracy: 0.6699
Epoch 333/500
1536/1536 [==============================] - 1s 457us/step - loss: 1.0989 - sparse_categorical_accuracy: 0.6719
Epoch 

1536/1536 [==============================] - 1s 464us/step - loss: 1.0292 - sparse_categorical_accuracy: 0.6923
Epoch 392/500
1536/1536 [==============================] - 1s 466us/step - loss: 1.0281 - sparse_categorical_accuracy: 0.6957
Epoch 393/500
1536/1536 [==============================] - 1s 459us/step - loss: 1.0289 - sparse_categorical_accuracy: 0.6926
Epoch 394/500
1536/1536 [==============================] - 1s 466us/step - loss: 1.0268 - sparse_categorical_accuracy: 0.6923
Epoch 395/500
1536/1536 [==============================] - 1s 458us/step - loss: 1.0228 - sparse_categorical_accuracy: 0.6970
Epoch 396/500
1536/1536 [==============================] - 1s 464us/step - loss: 1.0224 - sparse_categorical_accuracy: 0.6942
Epoch 397/500
1536/1536 [==============================] - 1s 467us/step - loss: 1.0197 - sparse_categorical_accuracy: 0.6979
Epoch 398/500
1536/1536 [==============================] - 1s 472us/step - loss: 1.0200 - sparse_categorical_accuracy: 0.6963
Epoch 

1536/1536 [==============================] - 1s 478us/step - loss: 0.9626 - sparse_categorical_accuracy: 0.7149
Epoch 457/500
1536/1536 [==============================] - 1s 477us/step - loss: 0.9640 - sparse_categorical_accuracy: 0.7125
Epoch 458/500
1536/1536 [==============================] - 1s 465us/step - loss: 0.9619 - sparse_categorical_accuracy: 0.7134
Epoch 459/500
1536/1536 [==============================] - 1s 459us/step - loss: 0.9606 - sparse_categorical_accuracy: 0.7126
Epoch 460/500
1536/1536 [==============================] - 1s 472us/step - loss: 0.9593 - sparse_categorical_accuracy: 0.7138
Epoch 461/500
1536/1536 [==============================] - 1s 477us/step - loss: 0.9600 - sparse_categorical_accuracy: 0.7140
Epoch 462/500
1536/1536 [==============================] - 1s 467us/step - loss: 0.9594 - sparse_categorical_accuracy: 0.7137
Epoch 463/500
1536/1536 [==============================] - 1s 467us/step - loss: 0.9572 - sparse_categorical_accuracy: 0.7145
Epoch 

In [279]:
def get_inference_model_complex(model):
    lstm1 = model.get_layer(name='lstm_1')
    lstm2 = model.get_layer(name='lstm_2')
    lstm3 = model.get_layer(name='lstm_3')
    dense = model.get_layer(name='dense_output_lstm')
    
    inp_infer = Input(shape=(1, feature_size))
    X = lstm1(inp_infer)
    X = lstm2(X)
    X = lstm3(X)
    X = dense(X)
    model_infer = Model(inp_infer, X)
    return model_infer

In [315]:
cmplx_inf_model = get_inference_model_complex(complx)

In [320]:
generate_name(cmplx_inf_model)

'pameeavuiorialiereui'